In [1]:
import os, pyperclip, re, subprocess, requests, json, random
from pymediainfo import MediaInfo
from datetime import datetime
import subprocess
from smms import *
from utils import *
crf=18.0
qcomp=0.7
aq_mode=3
aq_s=0.7
bitdepth=8
newline="\r\n"

PROCUCT_DEST_DIR=r"~/encode"
MOVIE=r"/home/qianyi/Downloads/The.Devil.Wears.Prada.2006.ENG.Blu-ray.1080p.REMUX.DTS-louco@PTH.mkv"

#https://api.rhilip.info/ptgen.html
DOUBAN_ID="1482072"

SRC="The.Devil.Wears.Prada.2006.ENG.Blu-ray.1080p.AVC.DTS-HD.MA5.1-TPKHD@PTHome"

REMUX_DIRNAME="The.Devil.Wears.Prada.2006.ENG.Blu-ray.1080p.AVC.REMUX.DTS-HD.MA5.1-louco@PTH"

ACKNOWLEDGEMENT='''原盘@DON，字幕素材@字幕库，感谢原发布者。
由PTHome字幕组@TPKHD完成校对、调整及特效处理。'''

SOUNDTRACK="【英语｜国配】【简|繁|简英|繁英|音字幕】"

SUBTITLE_GZ=1 #官字组

metainfo=getmetainfo(DOUBAN_ID)
smms = SMMS('hzlclock', 'Hezl1996')
smms.get_api_token()

In [4]:
# crop=subprocess.getoutput("ffmpeg -i "+MOVIE+" -t 1 -vf cropdetect -f null - 2>&1")
#ffmpeg -i $MOVIE -t 1 -vf cropdetect -f null - 2>&1 | awk '/crop/ { print $NF }' | tail -1
# try: crop="-filter:v \""+re.findall(r'crop=.*',crop)[-1]+"\""
# except: pass
MOVIE=r"/sdcard/The.Devil.Wears.Prada.2006.ENG.Blu-ray.1080p.REMUX.DTS-louco@PTH.mkv"
crop='-filter:v "crop=in_w:in_h-280"'
test=""
x264_stmt=f"x264 --demuxer y4m --preset slower --profile high --level 4.1 --ref 4 \
--crf {crf} --qcomp {qcomp} --aq-mode {aq_mode} --aq-strength {aq_s} --bframes 11 --me umh  --subme 11 \
--merange 48 --no-fast-pskip --no-dct-decimate --direct auto --psy-rd 1.00:0.00 \
--vbv-bufsize 78125 --vbv-maxrate 62500 --deblock -3:-3 --b-adapt 2 --keyint 240 \
--min-keyint 1 --no-mbtree --trellis 2 --chroma-qp-offset -1 --rc-lookahead 72 --output 264.mkv -"

x265_stmt=f"x265 --y4m -D 10 --profile main10 --level-idc 5.1 --high-tier --preset slower \
--crf {crf} --qcomp {qcomp} --aq-mode {aq_mode} --aq-strength {aq_s} \
--rd 4 --psy-rd 2.0 --psy-rdoq 1.0 --rdoq-level 2 --deblock -1:-1 --cbqpoffs -1 --crqpoffs -3 --ctu 32 \
--qg-size 8 --no-sao --no-sao-non-deblock --selective-sao 0 --tu-intra-depth 4 --tu-inter-depth 4 --no-opt-qp-pps \
--no-opt-ref-list-length-pps --aud --repeat-headers --me 3 --subme 5 --merange 48 --b-intra --limit-tu 0 --no-rect \
--no-amp --no-open-gop --pools + --keyint 240 --min-keyint 1 --bframes 8 --max-merge 4 --ref 4 --weightb --rc-lookahead 72 \
--scenecut 40 --no-strong-intra-smoothing --vbv-bufsize 160000 --vbv-maxrate 160000 --input-depth 10 --output 265.mkv -"

test="-ss 3900 -t 60"

args=x264_stmt.replace(" --","\n --").split("\n")[1:-1]
args=[i.strip().replace("--","").replace(" ","=").replace(":",",") for i in args]
args5=x265_stmt.replace(" --","\n --").split("\n")[1:-1]
args5=[i.strip().replace("--","").replace(" ","=").replace(":",",") for i in args5]

x264_filename="".join(re.findall(r'([^/\\]*$)', MOVIE)).replace("REMUX", "x264")
x264_dirname=x264_filename.replace(".mkv","")
x264_fullpath=os.path.join(PROCUCT_DEST_DIR, x264_dirname, x264_filename)
x265_filename="".join(re.findall(r'([^/\\]*$)', MOVIE)).replace("REMUX", "x265.10bit")
x265_dirname=x265_filename.replace(".mkv","")
x265_fullpath=os.path.join(PROCUCT_DEST_DIR, x265_dirname, x265_filename)

cmd264=f"mkdir -p {os.path.join(PROCUCT_DEST_DIR, x264_dirname)}\n"
cmd265=f"mkdir -p {os.path.join(PROCUCT_DEST_DIR, x265_dirname)}\n"

torrent264=f"transmission-create -p -o {PROCUCT_DEST_DIR}/{x264_dirname}.torrent -t https://www.pthome.net/announce.php \
{os.path.join(PROCUCT_DEST_DIR, x264_dirname)}"
torrent265=f"transmission-create -p -o {PROCUCT_DEST_DIR}/{x265_dirname}.torrent -t https://www.pthome.net/announce.php \
{os.path.join(PROCUCT_DEST_DIR, x265_dirname)}"

cmd264+=f"ffmpeg {test} -i {MOVIE} -c:v libx264 -profile:v high {crop} -preset slow -x264-params {':'.join(args)} \
-max_muxing_queue_size 1024 -c:a copy -c:s copy -max_muxing_queue_size 1024 -map 0 {x264_fullpath} 2>&1|\
tee {os.getcwd()}/x264.log\nmkvpropedit {x264_fullpath} --add-track-statistics-tags\n"
cmd264+=get_captures("x264",x264_fullpath)+"\n"
cmd264+=torrent264+"\n"

cmd265+=f"ffmpeg {test} -i {MOVIE} -c:v libx265 -profile:v main10 {crop} -preset slow -x265-params {':'.join(args5)} \
-max_muxing_queue_size 1024 -c:a copy -c:s copy -max_muxing_queue_size 1024 -map 0 {x265_fullpath} 2>&1|\
tee {os.getcwd()}/x265.log\nmkvpropedit {x265_fullpath} --add-track-statistics-tags\n"+get_captures("x265",x265_fullpath)
cmd265+=get_captures("x264",x265_fullpath)+"\n"
cmd265+=torrent265+"\n"

remux_fullpath=f"{PROCUCT_DEST_DIR}/{REMUX_DIRNAME}/{REMUX_DIRNAME}.mkv"
cmdremux=get_captures("REMUX",remux_fullpath)
cmdremux+=f"transmission-create -p -o {PROCUCT_DEST_DIR}/{REMUX_DIRNAME}.torrent -t https://www.pthome.net/announce.php \
{os.path.join(PROCUCT_DEST_DIR, REMUX_DIRNAME)}"

with open("x264.sh","w") as f:
    f.write(cmd264)
with open("x265.sh","w") as f:
    f.write(cmd265)
with open("remux.sh","w") as f:
    f.write(cmdremux)

In [9]:
encode_picbed=upload_folder("x264_captures")

===UPLOAD===
https://i.loli.net/2021/01/30/iO9CeUJkMxwKHtP.jpg
https://i.loli.net/2021/01/30/iBocqxOSepbCQtK.jpg
https://i.loli.net/2021/01/30/CsIQL1d7HhMmzEi.jpg
https://i.loli.net/2021/01/30/3XwMa7CQJ4uO1F6.jpg
https://i.loli.net/2021/01/30/TP8eYhDHiLQuk4r.jpg
https://i.loli.net/2021/01/30/PRAwCGiztb2MHYU.jpg


In [109]:
remux_picbed=upload_folder("REMUX_captures")

===UPLOAD===
https://i.loli.net/2021/02/01/URZ2pMP6HJ1xfYK.jpg
https://i.loli.net/2021/02/01/vKkpaPh5nDixVzd.jpg
https://i.loli.net/2021/02/01/LZFOUqRCA27gWH1.jpg
https://i.loli.net/2021/02/01/qr1mKAFVNejZanB.jpg
https://i.loli.net/2021/02/01/vKkpaPh5nDixVzd.jpg
https://i.loli.net/2021/02/01/mMC9TH6wsGAtDpP.jpg


In [140]:
with open(f"x264.bbcode.txt","w") as f:
    f.write(get_bbcode(x264_fullpath, src=SRC, logname="x264.log", picbed=encode_picbed, metainfo=metainfo))
with open(f"x265.bbcode.txt","w") as f:
    f.write(get_bbcode(x265_fullpath, src=SRC, logname="x265.log", picbed=encode_picbed, metainfo=metainfo))
with open(f"remux.bbcode.txt","w") as f:
    f.write(get_bbcode(remux_fullpath, src=SRC, picbed=encode_picbed, metainfo=metainfo))

Title: /home/qianyi/Downloads/The Boat 2018 1080p ENG Blu-ray x264 DTS-louco@PTH/The Boat 2018 1080p ENG Blu-ray x264 DTS-louco@PTH mkv
Title: /home/qianyi/Downloads/The Boat 2018 1080p ENG Blu-ray x265 10bit DTS-louco@PTH/The Boat 2018 1080p ENG Blu-ray x265 10bit DTS-louco@PTH mkv
Title: /home/qianyi/Downloads//The Boat 2018 1080p ENG Blu-ray AVC REMUX DTS-HD MA5 1-louco@PTH/The Boat 2018 1080p ENG Blu-ray AVC REMUX DTS-HD MA5 1-louco@PTH mkv


In [138]:
with open(f"x264.bbcode.base64.txt","w") as f:
    f.write(getbase64json(x264_fullpath, src=SRC, logname="x264.log", picbed=encode_picbed, metainfo=metainfo))
with open(f"x265.bbcode.base64.txt","w") as f:
    f.write(getbase64json(x265_fullpath, src=SRC, logname="x265.log", picbed=encode_picbed, metainfo=metainfo))
with open(f"remux.bbcode.base64.txt","w") as f:
    f.write(getbase64json(remux_fullpath, src=SRC, logname="", picbed=remux_picbed, metainfo=metainfo, is_remux=True))

Title: /home/qianyi/Downloads/The Boat 2018 1080p ENG Blu-ray x264 DTS-louco@PTH/The Boat 2018 1080p ENG Blu-ray x264 DTS-louco@PTH mkv
Title: /home/qianyi/Downloads/The Boat 2018 1080p ENG Blu-ray x265 10bit DTS-louco@PTH/The Boat 2018 1080p ENG Blu-ray x265 10bit DTS-louco@PTH mkv
Title: /home/qianyi/Downloads//The Boat 2018 1080p ENG Blu-ray AVC REMUX DTS-HD MA5 1-louco@PTH/The Boat 2018 1080p ENG Blu-ray AVC REMUX DTS-HD MA5 1-louco@PTH mkv
